In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import skimage
import skimage.transform
import skimage.filters
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda
import torch.optim as optim
import time
from tqdm.notebook import tqdm
from torch.nn.functional import relu
from sklearn.utils import shuffle
from SHG import SHG
from utils import *

In [2]:
TRAIN_LABELS_PATH = "D:/bsc_data/train/outputs.txt"
TEST_LABELS_PATH = "D:/bsc_data/test/outputs.txt"
VAL_LABELS_PATH = "D:/bsc_data/validation/outputs.txt"

HEADER = ["ID"]
for i in range(17):
    HEADER.append("x{}".format(i))
    HEADER.append("y{}".format(i))
    HEADER.append("v{}".format(i))

train_labels = pd.read_csv(TRAIN_LABELS_PATH, delimiter = ",", names = HEADER)
test_labels = pd.read_csv(TEST_LABELS_PATH, delimiter = ",", names = HEADER)
val_labels = pd.read_csv(VAL_LABELS_PATH, delimiter = ",", names = HEADER)

TRAIN_IMGS_PATH = "D:/bsc_data/train/image/"
TEST_IMGS_PATH = "D:/bsc_data/test/image/"
VAL_IMGS_PATH = "D:/bsc_data/validation/image/"

train_imgs = os.listdir(TRAIN_IMGS_PATH)
test_imgs = os.listdir(TEST_IMGS_PATH)
val_imgs = os.listdir(VAL_IMGS_PATH)

train_labels, train_imgs = shuffle(train_labels, train_imgs)
test_labels, test_imgs = shuffle(test_labels, test_imgs)
#val_labels, val_imgs = shuffle(val_labels, val_imgs) # NOGET ER GALT

In [3]:
LEARNING_RATE = 2.5e-4
NUM_EPOCHS = 100
MINI_BATCH_SIZE = 16
MINI_BATCHES = np.array_split(train_imgs, len(train_imgs)/MINI_BATCH_SIZE)
USE_SAVED_MODEL = False
SAVED_MODEL_PATH = None

try:
    average_rgb = np.loadtxt("./average_rgb.npy")
except:
    average_rgb = get_mean_rgb(TRAIN_IMGS_PATH, train_imgs)
    np.savetxt("./average_rgb.npy", average_rgb)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SHG(num_hourglasses=1).to(device)

if USE_SAVED_MODEL:
    model.load_state_dict(torch.load(SAVED_MODEL_PATH))

criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr = LEARNING_RATE)

In [4]:
cur_model_path = "./models/" + time.asctime().replace(" ", "_").replace(":", "-")
os.mkdir(cur_model_path)

In [6]:
torch.cuda.empty_cache()
model.train()
average_loss = []

for epoch in tqdm(range(NUM_EPOCHS), desc = "EPOCH"):
    average_loss.append(0)
    for mini_batch in tqdm(MINI_BATCHES, leave = False, desc = "MINI BATCH"):
        heatmaps = []
        imgs = []

        for img_name in tqdm(mini_batch, desc = "CREATING GT HEATMAPS", leave = False):
            heatmaps.append(create_heatmaps(train_labels.loc[train_labels["ID"] == img_name[:-4]].to_numpy()))
            img = plt.imread(TRAIN_IMGS_PATH + img_name)

            if (len(img.shape) == 2):
                img = grey_to_rgb(img)

            img -= average_rgb

            imgs.append(img)
        
        heatmaps = torch.FloatTensor(heatmaps).to(device)
        imgs = (torch.from_numpy(np.array(imgs)).permute(0, 3, 1, 2)).to(device)

        predictions = model(imgs)

        optimizer.zero_grad()

        loss = criterion(predictions.to(device), heatmaps)
        print(loss)

        average_loss[-1] += loss/MINI_BATCH_SIZE

        loss.backward()
        optimizer.step()
    
    # Save model
    torch.save(model.state_dict(), cur_model_path + "/epoch_{}_latest".format(epoch) + ".pth")

    # Saving loss
    np.savetxt(cur_model_path + "/loss.npy", average_loss)

    print("average loss of epoch {}: {}".format(epoch, average_loss[-1]))       



tensor(1.5309e-07, device='cuda:0', grad_fn=<MseLossBackward>)



tensor(1.5458e-07, device='cuda:0', grad_fn=<MseLossBackward>)



tensor(1.5485e-07, device='cuda:0', grad_fn=<MseLossBackward>)



tensor(1.5237e-07, device='cuda:0', grad_fn=<MseLossBackward>)



tensor(1.5455e-07, device='cuda:0', grad_fn=<MseLossBackward>)



tensor(1.5576e-07, device='cuda:0', grad_fn=<MseLossBackward>)



tensor(1.3020e-07, device='cuda:0', grad_fn=<MseLossBackward>)


KeyboardInterrupt: 